In [1]:
import os

db_S_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0/S/"
db_L_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0/L/"
db_M_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0/M/"
db_patterns_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0/patterns/"

model_dir_root = os.environ["DATA"] + "models/db3.0.0/01_protov5/"
shap_scale_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "shap_scale.png")
yes_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "button_yes.png")
no_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "button_no.png")
yes_small_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "button_yes_small.png")
no_small_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "button_no_small.png")
pos_pred_legend_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "cf_info_pos.png")
neg_pred_legend_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0", "cf_info_neg.png")
interface_dir = os.environ["DATA"] + "webinterfaces/int05_prototype/"

XAI_DATASET_SIZE = 50

N_JOBS = 20
N_JOBS_GPU = 4

RESNET_TYPE = "resnet50"

In [2]:

# Grid division for full image
X_DIVISIONS_L = 15
Y_DIVISIONS_L = 15
X_DIVISIONS_S = 10
Y_DIVISIONS_S = 10
X_DIVISIONS_M = 12
Y_DIVISIONS_M = 12

# Grid division of patterns
X_DIVISIONS_PATTERNS = 2
Y_DIVISIONS_PATTERNS = 2

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS = ["#A33E9A", "#E0B000", "#0C90C0"]  # Purple, Yellow, Blue

explict_colors_dict = {
    "#A33E9A": "purple",
    "#E0B000": "yellow",
    "#0C90C0": "blue"
}

In [3]:
from xaipatimg.datagen.dbimg import load_db

db_patterns = load_db(db_patterns_dir)

In [4]:
import numpy as np

pattern_3sym_2col_keys = []

# Extracting list of patterns that contain 3 symbols of 3 different shapes and 2 different colors. The two items of the same color cannot be
# on a diagonal.
for k, v in db_patterns.items():
    if len(v["content"]) == 3:
        img_col_d = {}
        img_shape_d = {}
        color_matrix = np.full((2, 2), "", dtype="U100")
        for entry in v["content"]:
            img_col_d[entry["color"]] = True
            img_shape_d[entry["shape"]] = True
            color_matrix[entry["pos"][0]][entry["pos"][1]] = entry["color"]

        same_color_on_diagonal = color_matrix[0][0] == color_matrix[1][1] or color_matrix[0][1] == color_matrix[1][0]

        if len(img_col_d.keys()) == 2 and len(img_shape_d.keys()) == 3 and not same_color_on_diagonal:
            pattern_3sym_2col_keys.append(k)

In [5]:
datasets_path_L = os.path.join(db_L_dir, "datasets", "01_protov5")
datasets_path_S = os.path.join(db_S_dir, "datasets", "01_protov5")
datasets_path_M = os.path.join(db_M_dir, "datasets", "01_protov5")

In [6]:
from xaipatimg.datagen.gendataset import generic_rule_pattern_exactly_1_time_exclude_more, \
    generic_rule_N_times_color_shape_exactly, generic_rule_shape_in_every_row

rules_data_L = [
    #
    # {"name": "hard1_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
    #                                                                                                  "y_division_full": Y_DIVISIONS_L,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True},
    #  "question": "Is the pattern or any of its left or right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[0]},
    #
    # {"name": "hard2_blue_circle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "circle",
    #                                                                                             "color": "#0C90C0",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of blue circles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "The image contains 13 blue circles. They are highlighted below. | A1;B1;C1;B2;C2;D2;E4;F4;G5;A6;B6;E6;D7", "neg_llm_scaffold": "The image contains 12 blue circles instead of 13. They are highlighted below. | A2;B3;D3;B3;C3;D3;E5;F6;A7;B7;E7;H9"},
    #
    # {"name": "hard2bis_blue_circle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "circle",
    #                                                                                             "color": "#0C90C0",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of blue circles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard3_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
    #                                                                                                  "y_division_full": Y_DIVISIONS_L,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True},
    #  "question": "Is the pattern or any of its left or right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[1]},

    # {"name": "hard4_purple_triangle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of purple triangles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard4bis_purple_triangle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                             "x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of purple triangles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "hard4_yellow_triangle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of yellow triangles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard4bis_yellow_triangle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                             "x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of yellow triangles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard4_yellow_square_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "square",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of yellow squares equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard4bis_yellow_square_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                             "x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "square",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of yellow squares equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard4_blue_circle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "circle",
    #                                                                                             "color": "#0C90C0",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of blue circles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard4bis_blue_circle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                             "x_division": X_DIVISIONS_L,
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "circle",
    #                                                                                             "color": "#0C90C0",
    #                                                                                             "N": 13,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of blue circles equal to 13 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    {"name": "hard5_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
                                                                                                     "y_division_full": Y_DIVISIONS_L,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "The pattern is found in the image and is highlighted below. | B5;C5;B6", "neg_llm_scaffold": "The pattern was not found in the image. |", "pattern_id": pattern_3sym_2col_keys[2]},
    #
    # {"name": "hard6_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
    #                                                                                                  "y_division_full": Y_DIVISIONS_L,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": False},
    #  "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[3]},

    # {"name": "hard7_square_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "square",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a square in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "hard8_square_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_L,
    #                                                                                             "shape": "square",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a square in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""}

]

In [ ]:
from xaipatimg.datagen.gendataset import generic_rule_pattern_exactly_1_time_exclude_more, \
    generic_rule_N_times_color_shape_exactly

rules_data_S = [
    #
    # {"name": "easy1_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
    #                                                                                                  "y_division_full": Y_DIVISIONS_S,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True},
    #  "question": "Is the pattern or any of its left or right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[4]},
    #
    {"name": "easy2_yellow_triangle_N", "gen_fun": generic_rule_N_times_color_shape_exactly,
     "gen_kwargs": {"x_division": X_DIVISIONS_S,
                    "y_division": Y_DIVISIONS_S,
                    "shape": "triangle",
                    "color": "#E0B000",
                    "N": 6,
                    "restrict_plus_minus_1": True},
     "question": "Does the number of yellow triangles equal to 6 in the image?", "target_acc": 0.85, "shown_acc": 0.85,
     "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "easy2bis_yellow_triangle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                             "x_division": X_DIVISIONS_S,
    #                                                                                             "y_division": Y_DIVISIONS_S,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 6,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of yellow triangles equal to 6 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "easy3_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
    #                                                                                                  "y_division_full": Y_DIVISIONS_S,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True},
    #  "question": "Is the pattern or any of its left or right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[5]},
    #
    {"name": "easy4_purple_circle_N", "gen_fun": generic_rule_N_times_color_shape_exactly,
     "gen_kwargs": {"x_division": X_DIVISIONS_S,
                    "y_division": Y_DIVISIONS_S,
                    "shape": "circle",
                    "color": "#A33E9A",
                    "N": 6,
                    "restrict_plus_minus_1": True},
     "question": "Does the number of purple circles equal to 6 in the image?", "target_acc": 0.85, "shown_acc": 0.85,
     "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "easy4bis_purple_circle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                               "x_division": X_DIVISIONS_S,
    #                                                                                               "y_division": Y_DIVISIONS_S,
    #                                                                                               "shape": "circle",
    #                                                                                               "color": "#A33E9A",
    #                                                                                               "N": 6,
    #                                                                                               "restrict_plus_minus_1": False},
    #  "question": "Does the number of purple circles equal to 6 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #

    {"name": "easy5_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more,
     "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
                    "y_division_full": Y_DIVISIONS_S,
                    "x_division_pattern": X_DIVISIONS_PATTERNS,
                    "y_division_pattern": Y_DIVISIONS_PATTERNS,
                    "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc": 0.85, "shown_acc": 0.85, "samples_interface": 13,
     "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[6]},
    #
    {"name": "easy6_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more,
     "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
                    "y_division_full": Y_DIVISIONS_S,
                    "x_division_pattern": X_DIVISIONS_PATTERNS,
                    "y_division_pattern": Y_DIVISIONS_PATTERNS,
                    "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc": 0.85, "shown_acc": 0.85, "samples_interface": 13,
     "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[7]},

    # {"name": "easy7_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
    #                                                                                                  "y_division_full": Y_DIVISIONS_S,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True},
    #  "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[10]},

    # {"name": "easy8_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
    #                                                                                                  "y_division_full": Y_DIVISIONS_S,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": False},
    #  "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[11]},
    #
    # {"name": "easy9_blue_square_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_S,
    #                                                                                               "y_division": Y_DIVISIONS_S,
    #                                                                                               "shape": "square",
    #                                                                                               "color": "#0C90C0",
    #                                                                                               "N": 6,
    #                                                                                               "restrict_plus_minus_1": True},
    #  "question": "Does the number of blue squares equal to 6 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "easy10_circle_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_S,
    #                                                                                             "shape": "circle",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a circle in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "easy11_circle_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_S,
    #                                                                                             "shape": "circle",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a circle in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "easy12_circle_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_S,
    #                                                                                             "shape": "circle",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a circle in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

]

In [ ]:
from xaipatimg.datagen.gendataset import generic_rule_pattern_exactly_1_time_exclude_more, \
    generic_rule_N_times_color_shape_exactly

rules_data_M = [

    # {"name": "med1_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_M,
    #                                                                                                  "y_division_full": Y_DIVISIONS_M,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True},
    #  "question": "Is the pattern or any of its left or right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[8]},
    #
    {"name": "med2_yellow_square_N", "gen_fun": generic_rule_N_times_color_shape_exactly,
     "gen_kwargs": {"x_division": X_DIVISIONS_M,
                    "y_division": Y_DIVISIONS_M,
                    "shape": "square",
                    "color": "#E0B000",
                    "N": 8,
                    "restrict_plus_minus_1": True},
     "question": "Does the number of yellow squares equal to 8 in the image?", "target_acc": 0.85, "shown_acc": 0.85,
     "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    #
    # {"name": "med2bis_yellow_square_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "square",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of yellow squares equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    #
    #
    # {"name": "med2_purple_triangle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of purple triangles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "med2_purple_square_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "square",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of purple squares equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "med2bis_purple_square_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "square",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of purple squares equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "med2_yellow_triangle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of yellow triangles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    #
    # {"name": "med2bis_yellow_triangle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#E0B000",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of yellow triangles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "med2_purple_circle_N", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "circle",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": True},
    #  "question": "Does the number of purple circles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    #
    # {"name": "med2bis_purple_circle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "circle",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of purple circles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    #
    # {"name": "med2bis_purple_triangle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {"x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#A33E9A",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of purple triangles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},

    # {"name": "med3_find_pattern_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_M,
    #                                                                                                  "y_division_full": Y_DIVISIONS_M,
    #                                                                                                  "x_division_pattern": X_DIVISIONS_PATTERNS,
    #                                                                                                  "y_division_pattern": Y_DIVISIONS_PATTERNS,
    #                                                                                                  "consider_rotations": True,
    #                                                                                                         },
    #  "question": "Is the pattern or any of its left or right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[9]},
    #
    {"name": "med4_blue_triangle_N", "gen_fun": generic_rule_N_times_color_shape_exactly,
     "gen_kwargs": {"x_division": X_DIVISIONS_M,
                    "y_division": Y_DIVISIONS_M,
                    "shape": "triangle",
                    "color": "#0C90C0",
                    "N": 8,
                    "restrict_plus_minus_1": True},
     "question": "Does the number of blue triangles equal to 8 in the image?", "target_acc": 0.85, "shown_acc": 0.85,
     "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "med4bis_blue_triangle_N_norestrict", "gen_fun": generic_rule_N_times_color_shape_exactly, "gen_kwargs": {
    #                                                                                             "x_division": X_DIVISIONS_M,
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "color": "#0C90C0",
    #                                                                                             "N": 8,
    #                                                                                             "restrict_plus_minus_1": False},
    #  "question": "Does the number of blue triangles equal to 8 in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    {"name": "med5_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more,
     "gen_kwargs": {"x_division_full": X_DIVISIONS_M,
                    "y_division_full": Y_DIVISIONS_M,
                    "x_division_pattern": X_DIVISIONS_PATTERNS,
                    "y_division_pattern": Y_DIVISIONS_PATTERNS,
                    "consider_rotations": False,
                    },
     "question": "Is the pattern in the image?", "target_acc": 0.85, "shown_acc": 0.85, "samples_interface": 13,
     "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[10]},

    {"name": "med6_find_pattern", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more,
     "gen_kwargs": {"x_division_full": X_DIVISIONS_M,
                    "y_division_full": Y_DIVISIONS_M,
                    "x_division_pattern": X_DIVISIONS_PATTERNS,
                    "y_division_pattern": Y_DIVISIONS_PATTERNS,
                    "consider_rotations": False,
                    },
     "question": "Is the pattern in the image?", "target_acc": 0.85, "shown_acc": 0.85, "samples_interface": 13,
     "pos_llm_scaffold": "", "neg_llm_scaffold": "", "pattern_id": pattern_3sym_2col_keys[11]},

    #
    # {"name": "med7_triangle_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a triangle in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
    #
    # {"name": "med8_triangle_allrows", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {
    #                                                                                             "y_division": Y_DIVISIONS_M,
    #                                                                                             "shape": "triangle",
    #                                                                                             "exclude_two_rows_missing": True},
    #  "question": "Is there a triangle in every row (1, 2, ...) of the image ?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 13, "pos_llm_scaffold": "", "neg_llm_scaffold": ""},
]


In [7]:
from xaipatimg.ml.xai import generate_shap_resnet, generate_counterfactuals_resnet_random_approach, \
    create_xai_index
from tqdm import tqdm


def generate_explanations(rules_data, db_dir, datasets_dir_path):
    for rule_idx in tqdm(range(len(rules_data))):

        model_dir = os.path.join(model_dir_root, rules_data[rule_idx]["name"])
        dataset_filename = rules_data[rule_idx]["name"] + "_test.csv"
        generic_rule_fun = rules_data[rule_idx]["gen_fun"]
        generic_rule_fun_kwargs = rules_data[rule_idx]["gen_kwargs"]
        xai_output_paths = {
            "shap": "shap",
            "cf": "cf",
        }

        if "pattern_id" in rules_data[rule_idx]:
            generic_rule_fun_kwargs["pattern_content"] = db_patterns[rules_data[rule_idx]["pattern_id"]]["content"]

        generate_shap_resnet(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
                             model_dir=model_dir, xai_output_path=os.path.join(model_dir, xai_output_paths["shap"]),
                             yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path, device="cuda:0",
                             n_jobs=N_JOBS,
                             dataset_size=XAI_DATASET_SIZE, masker="ndarray", shap_scale_img_path=shap_scale_img_path,
                             resnet_type=RESNET_TYPE)

        generate_counterfactuals_resnet_random_approach(db_dir, datasets_dir_path=datasets_dir_path,
                                                        dataset_filename=dataset_filename,
                                                        model_dir=model_dir,
                                                        xai_output_path=os.path.join(model_dir, xai_output_paths["cf"]),
                                                        yes_pred_img_path=yes_pred_img_path,
                                                        no_pred_img_path=no_pred_img_path,
                                                        shapes=SHAPES, colors=COLORS, empty_probability=1 - SHAPE_PROB,
                                                        max_depth=10, nb_tries_per_depth=2000,
                                                        generic_rule_fun=generic_rule_fun,
                                                        devices=["cuda:0", "cuda:1"], n_jobs=N_JOBS_GPU,
                                                        dataset_size=XAI_DATASET_SIZE,
                                                        pos_pred_legend_path=pos_pred_legend_path,
                                                        neg_pred_legend_path=neg_pred_legend_path,
                                                        **generic_rule_fun_kwargs, resnet_type=RESNET_TYPE)

        create_xai_index(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
                         model_dir=model_dir,
                         xai_dirs=xai_output_paths, dataset_size=XAI_DATASET_SIZE, device="cuda:0",
                         resnet_type=RESNET_TYPE)


/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
generate_explanations(rules_data_S, db_S_dir, datasets_path_S)

NameError: name 'rules_data_S' is not defined

In [ ]:
# generate_explanations(rules_data_L, db_L_dir, datasets_path_L)

In [ ]:
# generate_explanations(rules_data_M, db_M_dir, datasets_path_M)

In [7]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import csv
from xaipatimg.ml.xai import generate_LLM_explanations, create_xai_index
from tqdm import tqdm

model_id = "openai/gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
)


def gen_LLM_explanations(db_dir, rules_data, datasets_dir_path, X_divisions, Y_divisions):
    db = load_db(db_dir)

    for rule_idx in tqdm(range(len(rules_data))):
        model_dir = os.path.join(model_dir_root, rules_data[rule_idx]["name"])
        dataset_filename = rules_data[rule_idx]["name"] + "_test.csv"

        # Extracting the subset of indices of samples selected for the experimental interface, in order to ease the cost of calculation
        interface_content_path = os.path.join(interface_dir, "res", "tasks",
                                              f"{rules_data[rule_idx]["name"]}_content.csv")
        interface_selected_idx = [int(row["og_idx"]) for row in
                                  list(csv.DictReader(open(interface_content_path), delimiter=','))]

        xai_output_paths = {
            "shap": "shap",
            "cf": "cf",
            "llm": "llm",
        }
        generate_LLM_explanations(db_dir, db, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
                                  model_dir=model_dir, llm_model=llm_model, llm_tokenizer=tokenizer,
                                  xai_output_path=os.path.join(model_dir, xai_output_paths["llm"]),
                                  explicit_colors_dict=explict_colors_dict, question=rules_data[rule_idx]["question"],
                                  yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path,
                                  yes_pred_img_path_small=yes_small_pred_img_path,
                                  no_pred_img_path_small=no_small_pred_img_path,
                                  X_division=X_divisions, Y_division=Y_divisions,
                                  device="cuda:0", dataset_size=XAI_DATASET_SIZE, only_for_index=interface_selected_idx,
                                  path_to_counterfactuals_dir_for_model_errors=os.path.join(model_dir,
                                                                                            xai_output_paths["cf"]),
                                  pos_llm_scaffold=rules_data[rule_idx]["pos_llm_scaffold"],
                                  neg_llm_scaffold=rules_data[rule_idx]["neg_llm_scaffold"],
                                  pattern_dict=db_patterns[rules_data[rule_idx]["pattern_id"]]["content"] if "pattern_id" in
                                                                                                             rules_data[rule_idx] else None,
                                  resnet_type=RESNET_TYPE)

        create_xai_index(db_dir, dataset_filename=dataset_filename, datasets_dir_path=datasets_dir_path,
                         model_dir=model_dir,
                         xai_dirs=xai_output_paths, dataset_size=XAI_DATASET_SIZE, device="cuda:0",
                         resnet_type=RESNET_TYPE)


/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.26s/it]


In [ ]:
gen_LLM_explanations(db_L_dir, rules_data_L, datasets_path_L, X_divisions=X_DIVISIONS_L, Y_divisions=Y_DIVISIONS_L)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/docker/data/models/db3.0.0/01_protov5/hard5_find_pattern/llm
Loading dataset content for hard5_find_pattern_test.csv



100%|██████████| 50/50 [00:00<00:00, 63.96it/s]
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)

  0%|          | 0/12 [00:00<?, ?it/s]

You are the explainability system of an AI model. Your role is to justify the decisions of the model. The role of the model is to answer questions about the content of images of symbols of colors. The images are described in a JSON data structure. The coordinates system uses letters from A to F for the columns and numbers from 1 to 6 for the rows. The user will provide you the prediction of the AI model for a given image and the corresponding JSON data. You need to give an explanation of the prediction. The explanation is expected to be a very short sentence which introduces a list of all coordinates that are involved in the model's prediction. The justification sentence and the list of coordinates must be separated by the character '|'. The coordinates are separated with the symbol ';', and there is no need to sort them. Do not use escape characters or markdown syntax. The question the model must answer is 'Is the pattern in the image?'.The pattern to search for is [{'shape': 'circle'


  8%|▊         | 1/12 [02:02<22:25, 122.35s/it]

...

<|channel|>analysis<|message|>We need to provide explanation for the AI's prediction "Yes" whether pattern is found in image. The pattern consists of three items:

- circle at A2, purple
- triangle at B1, blue
- square at B2, purple

Check whether these are present in the image list. Let's scan:

We see in the list: {'shape': 'circle', 'pos': 'A2', 'color': 'blue'}. That is circle at A2 but color blue, not purple. So the first pattern element does not match color. So pattern is not found. However there is no purple circle at A2; color mismatch.

Thus answer should be negative. Provide justifications. For negative: format: "The pattern was not found in the image. |" with nothing else after pipe. That matches the example for negative. The question: "Is the pattern in the image?" The model predicted Yes but that's wrong. We as explainability should give justification. They want us to provide a very short sentence that introduces a list of all coordinates involved in the model's prediction


 42%|████▏     | 5/12 [07:17<10:44, 92.09s/it]

<|channel|>analysis<|message|>We need to produce explanation with list of coordinates involved in the model's prediction. The model predicts "Yes" for the image, meaning pattern found. Pattern is defined: [{'shape': 'circle', 'pos': 'A2', 'color': 'purple'}, {'shape': 'triangle', 'pos': 'B1', 'color': 'blue'}, {'shape': 'square', 'pos': 'B2', 'color': 'purple'}].

We need to identify coordinates that match these pattern elements in the given data. Must list coordinates that are involved in the prediction, i.e., presumably the positions of circle at A2, triangle at B1, square at B2. Check if these exist.

In the data: there is no A2 entry? Let's search: I saw circle at A1, A5, A6, etc. No A2.

But perhaps the pattern means shape, pos, color. The coordinates pos must match pattern, but if any missing then pattern not found. But AI predicted Yes, so we assume there is such pattern present or model over predicted? But we need explain justification: list all coordinates involved in model's 


 50%|█████     | 6/12 [08:50<09:14, 92.40s/it]

<|channel|>analysis<|message|>We need to generate a very short sentence explaining the justification and list coordinates. The model predicted Yes for the pattern: circle at A2 purple, triangle at B1 blue, square at B2 purple. We need to check if that pattern exists in the provided image data.

The image has many shapes. We check: circle at A2? No, the image only has shapes at certain coordinates.

Let's parse the data quickly: It's long. Find A2: there is no A2; shapes are at A3, A5, A7, A9, A11, A12, A14, etc. So no circle at A2. So pattern not found. Wait? The pattern to search for: circle pos A2 purple; triangle B1 blue; square B2 purple. Let's see if each of these exists:

- circle A2 purple: not present.
- triangle B1 blue: there is no B1? Actually B1 is a square purple, not triangle. So pattern not found.

Given that the model predicted Yes, the explanation should indicate pattern found? However correct answer is negative? But we are the "explainability system" that justifies th